# <font color=red>LangChain:  Demos Planning and Solving Escape Room Puzzles </font>
- https://docs.langchain.com/docs

<span style="font-family:'Comic Sans MS', cursive, sans-serif;"><font color=orange>
## Demo 1 - Just Solving Puzzles (no planning)
</font></span>
This version is pretty in-expensive to run because there is no planning, puzzles are solved in pre-determined order.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import load_tools, AgentType, initialize_agent
from langchain.memory import ConversationBufferMemory
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

llm = ChatOpenAI(model="gpt-4", temperature=0.0, max_tokens=128)
tools = load_tools(["llm-math", "serpapi"], llm=llm)
memory = ConversationBufferMemory(memory_key="chat_history")

agent = initialize_agent(llm=llm,
                         tools=tools,
                         memory=memory,
                         agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
                         max_iterations=3,
                         verbose=True,)

SYSTEM_MSG = """
You are currently in a building that has 4 rooms.
The rooms are connected in sequence, so that you can go
from room 1 to room 2 to room 3 to room 4.
Room 4 room is where the exit for the building is located.
Each room has a puzzle that you must solve in order to move to the next room.
You will be able to leave the building if you get to room 4 and solve its puzzle.
You should solve the puzzle for each room that you are in, and then proceed to the next room.
When you solve a puzzle, state the solution so that your environment can be updated.
"""

USER_MSG = """
Here is info about about the current room your are in and the puzzle there:
ROOM:   {room}
PUZZLE: {puzzle}
{question}
"""

QUESTION = """
What is the solution to the puzzle in this room?
"""

puzzle1 = """
    This puzzle requires you to figure out the next value in the sequence:
        1, 4, 9, 16, 25, 36
    What is the next value?
"""

puzzle2 = """
    This puzzle is one that Caesar himself would have been proud of.
    This character string is an encrypted greeting:
        KHOOR
    Tell what the greeting is.
"""

puzzle3 = """
    Figure out how to place some books on a bookshelf by category.
    The bookshelf has these 5 labels:
        - Science
        - Fiction
        - Biography
        - Pets
        - Plants
    These are the books to be placed:
        - My Dog Chet
        - Life of Winston Churchill
        - Astronomy for Beginners
        - How to Grow Petunias
        - Sally and Tommy Visit the Zoo
    Tell which book goes in each category.
    Print each one as "category: book", e.g. Dogs: Fun With Dogs
"""

puzzle4 = """
    Next to the exit door, there is a keypad that accepts a 5-digit code.
    There is a note attached to the keypad that says:
        The number of words provides the clues.
    There is a piece of paper on the table that has these lines of text:
        - four grains of sand
        - peanut butter and jelly sandwich
        - some blue marbles
        - chocolate
        - candy in a brown paper sack
    Tell the order in which the keys must be pressed, e.g. 54321
"""

puzzles = [None, puzzle1, puzzle2, puzzle3, puzzle4]

def check_solution(room,solution):
    if current_room == 1:
        if "49" in output:
            return True
        else:
            return False
    elif current_room == 2:
        if "HELLO" in output  or  "hello" in output:
            return True
        else:
            return False
    elif current_room == 3:
        if  "Science: Astronomy" in output \
        and "Pets: My" in output \
        and "Biography: Life" in output \
        and "Plants: How" in output \
        and "Fiction: Sally" in output:
            return True
        else:
            return False
    elif current_room == 4:
        if "45316":
            return True
        else:
            return False
    print("**** INVALID ROOM ID",room)
    exit(-1)


current_room   = 1
current_puzzle = puzzles[current_room]

system_msg_template = SYSTEM_MSG
system_msg_prompt = SystemMessagePromptTemplate.from_template(system_msg_template)
user_template = USER_MSG
user_msg_prompt = HumanMessagePromptTemplate.from_template(user_template)
chat_prompt = ChatPromptTemplate.from_messages( [system_msg_prompt,user_msg_prompt] )

while True:
    prompt = chat_prompt.format(room=current_room, puzzle=current_puzzle, question=QUESTION)
    response = agent(prompt)
    output = response["output"]
    print(output)
    if not check_solution(current_room,output):
        print("**** OOPS: bad solution for room",current_room)
        break
    if current_room >= 4:
        print("EXIT SUCCESSFUL")
        break
    current_room += 1
    current_puzzle = puzzles[current_room]

<span style="font-family:'Comic Sans MS', cursive, sans-serif;"><font color=orange>
## Demo 2 - Planning which order to solve puzzles, and then solving them
</font></span>
This version can get expensive to run since it does a combination of planning and solving puzzles.</br>

Some things to note:
+ We use a <font color=green>StructuredTool</font> to retrieve info from a website about restrictions for order to solve puzzles:
  + https://bioseed.mcs.anl.gov/~rbutler/puzzle_info.html &nbsp;&nbsp; # https://www.cs.mtsu.edu/~rbutler/puzzle_info.html
+ We use <font color=green>ConversationSummaryBufferMemory</font> (note Summary) because the history becomes too large for the context window size
+ We use <font color=lightgreen>handle_parsing_errors=True</font> when initializing the agent because LangChain sometimes
has trouble parsing its own output; this argument usually mitigates that problem, but if not, you have to re-run the program

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import load_tools, AgentType, initialize_agent
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# from langchain.memory import ConversationBufferMemory
# ConversationSummaryMemory keeps a summary of the earliest pieces of conversation
#   while retaining a raw recollection of the latest interactions
from langchain.chains.conversation.memory import ConversationSummaryBufferMemory

from langchain.utilities import TextRequestsWrapper
from langchain.tools import StructuredTool

## import langchain
## langchain.debug = True  ######################################

requests = TextRequestsWrapper()
requests_get = StructuredTool.from_function(requests.get)

solved_puzzles = set()

llm = ChatOpenAI(model="gpt-4", temperature=0.0, max_tokens=128)
tools = load_tools(["llm-math", "serpapi"], llm=llm)
tools += [requests_get]
## memory = ConversationBufferMemory(memory_key="chat_history")
memory = ConversationSummaryBufferMemory(llm=llm,memory_key="chat_history")

agent = initialize_agent(llm=llm,
                         tools=tools,
                         memory=memory,
                         agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
                         max_iterations=3,
                         verbose=True,
                         handle_parsing_errors=True, # handle issue in LC
                        )

INITIAL_SYSTEM_MSG = """
You are in an escape room.
There are 4 puzzles in the room named:  puzzle_A, puzzle_B, puzzle_C, puzzle_D
You must solve all 4 puzzles in order to escape the room.
You must plan the order in which you wish to solve the puzzles.
There are restrictions on the order to solve the puzzles, described on this webpage:
    https://bioseed.mcs.anl.gov/~rbutler/puzzle_info.html
We will operate in a loop like this until all puzzles are solved:
    I will ask you to make a plan about which puzzle to solve next.
    You will state which puzzle you plan to solve next.
    I will ask you to solve that puzzle.
    You will state your solution to the puzzle.
"""
# https://www.cs.mtsu.edu/~rbutler/puzzle_info.html

INITIAL_USER_MSG = """
{instructions}
"""

PLAN_SOLVE_SYSTEM_MSG = """
"""

PLAN_SOLVE_USER_MSG = """
Please answer this question:
{question}
"""

puzzles = {}

puzzles["puzzle_A"] = """
    This puzzle is one that Caesar himself would have been proud of.
    This character string is an encrypted greeting:
        KHOOR
    Tell what the greeting is.
"""

puzzles["puzzle_B"] = """
    Figure out how to place some books on a bookshelf by category.
    The bookshelf has these 5 labels:
        - Science
        - Fiction
        - Biography
        - Pets
        - Plants
    These are the books to be placed:
        - My Dog Chet
        - Life of Winston Churchill
        - Astronomy for Beginners
        - How to Grow Petunias
        - Sally and Tommy Visit the Zoo
    Tell which book goes in each category.
    It is important to print each one as "category: book", e.g. Dogs: Fun With Dogs
"""

puzzles["puzzle_C"] = """
    There is a keypad that accepts a 5-digit code.
    There is a note attached to the keypad that says:
        The number of words provides the clues.
    There is a piece of paper on the table that has these lines of text:
        - four grains of sand
        - peanut butter and jelly sandwich
        - some blue marbles
        - chocolate
        - candy in a brown paper sack
    Tell the order in which the keys must be pressed, e.g. 54321
"""


puzzles["puzzle_D"] = """
    This puzzle requires you to figure out the next value in the sequence:
        1, 4, 9, 16, 25, 36
    What is the next value?
"""

def plan_is_ok(puzzle):
    global solved_puzzles
    if puzzle == "puzzle_D":
        if  len(solved_puzzles) == 0:
            return True
        else:
            print("FAILED PLAN FOR", puzzle, solved_puzzles)
            return False
    elif puzzle == "puzzle_C":
        if "puzzle_C" not in solved_puzzles and \
           "puzzle_A" not in solved_puzzles and \
           "puzzle_D" in solved_puzzles:
            return True
        else:
            print("FAILED PLAN FOR", puzzle, solved_puzzles)
            return False
    elif puzzle == "puzzle_A":
        if  "puzzle_A" not in solved_puzzles and \
            "puzzle_C" in solved_puzzles and \
            "puzzle_D" in solved_puzzles:
            return True
        else:
            print("FAILED PLAN FOR", puzzle, solved_puzzles)
            return False
    elif puzzle == "puzzle_B":
        if "puzzle_B" not in solved_puzzles  and  "puzzle_D" in solved_puzzles:
            return True
        else:
            print("FAILED PLAN FOR", puzzle, solved_puzzles)
            return False
    print("FAILED PLAN FOR PUZZLE", puzzle, solved_puzzles)
    return False

def solution_is_ok(puzzle,solution):
    if puzzle == "puzzle_D":
        if "49" in output:
            return True
        else:
            return False
    elif puzzle == "puzzle_A":
        if "HELLO" in output  or  "hello" in output:
            return True
        else:
            return False
    elif puzzle == "puzzle_B":
        if  "Science: Astronomy" in output \
        and "Pets: My" in output \
        and "Biography: Life" in output \
        and "Plants: How" in output \
        and "Fiction: Sally" in output:
            return True
        else:
            return False
    elif puzzle == "puzzle_C":
        if "45316" in solution:
            return True
        else:
            return False
    print("**** INVALID PUZZLE ID",puzzle)
    exit(-1)


# pre-planning
system_msg_template = INITIAL_SYSTEM_MSG
system_msg_prompt = SystemMessagePromptTemplate.from_template(system_msg_template)
user_template = INITIAL_USER_MSG
user_msg_prompt = HumanMessagePromptTemplate.from_template(user_template)
chat_prompt = ChatPromptTemplate.from_messages( [system_msg_prompt,user_msg_prompt] )
plan_instructions = "Please prepare to do planning and solve puzzles."
prompt = chat_prompt.format(instructions=plan_instructions)
response = agent(prompt)
output = response["output"]
print(output)

# prepare for questions about planning and obtaining solutions
system_msg_template = PLAN_SOLVE_SYSTEM_MSG
system_msg_prompt = SystemMessagePromptTemplate.from_template(system_msg_template)
user_template = PLAN_SOLVE_USER_MSG
user_msg_prompt = HumanMessagePromptTemplate.from_template(user_template)
chat_prompt = ChatPromptTemplate.from_messages( [system_msg_prompt,user_msg_prompt] )

while True:
    plan_query = """
        Given the current state, which puzzle should we solve next?
        Be sure to not select a puzzle which we have already solved.
        Please answer with:  CURRENT PUZZLE:
    """
    prompt = chat_prompt.format(question=plan_query)
    response = agent(prompt)
    output = response["output"]
    print(output)
    idx = output.find("CURRENT PUZZLE:")
    current_puzzle = output[idx+16:24]
    if current_puzzle[0:7] != "puzzle_" \
    or not plan_is_ok(current_puzzle):
        print("**** OOPS: bad plan for",current_puzzle)
        break
    puzzle_query = f"""
        Puzzle {current_puzzle} is defined as:
            {puzzles[current_puzzle]}
        What is the solution to that puzzle?
    """
    prompt = chat_prompt.format(question=puzzle_query)
    response = agent(prompt)
    output = response["output"]
    print(output)
    if not solution_is_ok(current_puzzle,output):
        print("**** OOPS: bad solution to puzzle",current_puzzle)
        break
    solved_puzzles.add(current_puzzle)
    if len(solved_puzzles) >= 4:
        print("EXIT SUCCESSFUL")
        break